# Creating the Database
---

Citi Bike first launched in New York City back in May of 2013 with 6,000 bikes total.  Since then, it has grown to 12,000 bikes shared among 143,000 members (as of May 2018).  Citi Bike's makes its data available for all.  Given the availability of said data, I plan on doing the following:

1. Build a SQL database populated with the data
2. Build visualization tools that would help tell a story
3. Build a time series model that would help with estimating future data

For this project, we will be using all available Citi Bike Jersey City trip data through December 31st, 2019.  The data can be found [here](https://s3.amazonaws.com/tripdata/index.html).  We will start off by building a webscraper to pull all the necessary data.  After downloading the data, we will combine them into a PostgreSQL database, which we will be able to query in a later notebook in order to extract any relevant data needed.

---
## Table of Contents

- [01. Importing Libraries](#01.-Importing-Libraries)
- [02. Data Prep](#02.-Data-Prep)
- [03. Building a PostgreSQL Database](#03.-Building-a-PostgreSQL-Database)

---
### 01. Importing Libraries

For this notebook, we will need the following libraries:

In [13]:
# Web scraping libraries
import requests
import urllib.request
from bs4 import BeautifulSoup

# Downloading, moving and unzipping files
import webbrowser
from time import sleep
import shutil
import os
from zipfile import ZipFile

# DataFrame exploration and manipulation
import pandas as pd
from glob import glob

# PostgreSQL interaction
import psycopg2
from psycopg2 import sql
from psycopg2 import Error
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

---
### 02. Data Prep

In the initial attempt, the link where all the data files were stored, https://s3.amazonaws.com/tripdata/index.html, returns a page which we cannot work with.  After some digging around and manipulating the url link, we ended up omitting the trailing 'index.html' piece to return a more workable page format.

In [2]:
url = 'https://s3.amazonaws.com/tripdata/'
response = requests.get(url)
response

<Response [200]>

A response of 200 is the greenlight we need to proceed.  Next we will instantiate our `soup` variable to look for the tag that we should leverage to obtain the links to each downloadable zip file.

In [3]:
soup = BeautifulSoup(response.text, 'xml')
# soup

Looking through what was returned above (as well as plugging in the url into a web browser, we can see that the .zip files that we want are stored with a `<Key>` tag.  So now we know where to look.

In [4]:
data_files = soup.find_all('Key')
# data_files

We're going to retreive all of the zip files from the site.  We will do this by first instantiating an empty list in order to store the proper file names found in `data_files`.  Afterwards, we will iterate through each list item and leverage the `webbrowser` library to open each respective download link.  This process runtime depends on how long the `sleep` function is used for.

In [5]:
# Instantiate empty list
zip_files = []

# Populate list with zip file names
for file in range(len(data_files)-1):
    zip_files.append(data_files[file].get_text())

# Download Jersey City zip files
for file in zip_files:
    if 'JC' in file:
        webbrowser.open_new(url + file)
        sleep(7)

After downloading all of the respective files, we will unzip their contents and then relocate them from the `/Downloads` folder to the `../data` of our project.

In [6]:
source = 'C:/Users/Fausto/Downloads/'
destination = 'C:/Users/Fausto/Documents/github_fmanon/Citibike_Trip_Data/data/'

# Unzip files and clean up data folder
for item in os.listdir(source):
    if item.endswith('.zip'):
        file_name = source + item
        zip_ref = ZipFile(file_name)
        zip_ref.extractall(source)
        zip_ref.close()
        os.remove(file_name)

# Move from Download folder to data folder
for item in os.listdir(source):
        shutil.move(source + item, destination)

Next, we'll take a look at details for some files, just to make sure everything lines up.

In [7]:
df = pd.read_csv('../data/JC-201509-citibike-tripdata.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6668 entries, 0 to 6667
Data columns (total 15 columns):
Trip Duration              6668 non-null int64
Start Time                 6668 non-null object
Stop Time                  6668 non-null object
Start Station ID           6668 non-null int64
Start Station Name         6668 non-null object
Start Station Latitude     6668 non-null float64
Start Station Longitude    6668 non-null float64
End Station ID             6668 non-null int64
End Station Name           6668 non-null object
End Station Latitude       6668 non-null float64
End Station Longitude      6668 non-null float64
Bike ID                    6668 non-null int64
User Type                  6668 non-null object
Birth Year                 5292 non-null float64
Gender                     6668 non-null int64
dtypes: float64(5), int64(5), object(5)
memory usage: 781.5+ KB


In [8]:
df2 = pd.read_csv('../data/JC-201912-citibike-tripdata.csv')
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19728 entries, 0 to 19727
Data columns (total 15 columns):
tripduration               19728 non-null int64
starttime                  19728 non-null object
stoptime                   19728 non-null object
start station id           19728 non-null int64
start station name         19728 non-null object
start station latitude     19728 non-null float64
start station longitude    19728 non-null float64
end station id             19728 non-null int64
end station name           19728 non-null object
end station latitude       19728 non-null float64
end station longitude      19728 non-null float64
bikeid                     19728 non-null int64
usertype                   19728 non-null object
birth year                 19728 non-null int64
gender                     19728 non-null int64
dtypes: float64(4), int64(6), object(5)
memory usage: 2.3+ MB


As you can see above, despite the columns being similar in name and data types, we need to rename all of the columns to match the same format.  This is because in order for `panda`'s concatenate method to work properly, all columns should be in the same format.

In [9]:
files = os.listdir('..//data/')

for csv in files:
    df = pd.read_csv(f'../data/{csv}')
    df = df.rename(columns=({'Trip Duration':'tripduration',
                             'Start Time':'starttime',
                             'Stop Time':'stoptime',
                             'Start Station ID':'start station id',
                             'Start Station Name':'start station name',
                             'Start Station Latitude':'start station latitude',
                             'Start Station Longitude':'start station longitude',
                             'End Station ID':'end station id',
                             'End Station Name':'end station name',
                             'End Station Latitude':'end station latitude',
                             'End Station Longitude':'end station longitude',
                             'Bike ID':'bikeid',
                             'User Type':'usertype',
                             'Birth Year':'birth year',
                             'Gender':'gender'}))
    df.to_csv(f'../data/{csv}', index = None)

Now that we have formatted all column headers across all downloaded data, we will now merge all `.csv` files into one for each city.

In [10]:
jc_files = sorted(glob('../data/JC-*******citibike-tripdata.csv'))
jc_trip_data = pd.concat((pd.read_csv(file) for file in jc_files), ignore_index = True)
jc_trip_data.to_csv('../data/jc_trip_data.csv', index = False)

for items in jc_files:
    os.remove(items)

Taking a quick look at the data we just combined.

In [11]:
jc_trip_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1403216 entries, 0 to 1403215
Data columns (total 15 columns):
tripduration               1403216 non-null int64
starttime                  1403216 non-null object
stoptime                   1403216 non-null object
start station id           1403216 non-null int64
start station name         1403216 non-null object
start station latitude     1403216 non-null float64
start station longitude    1403216 non-null float64
end station id             1403216 non-null int64
end station name           1403216 non-null object
end station latitude       1403216 non-null float64
end station longitude      1403216 non-null float64
bikeid                     1403216 non-null int64
usertype                   1402719 non-null object
birth year                 1358974 non-null float64
gender                     1403216 non-null int64
dtypes: float64(5), int64(5), object(5)
memory usage: 160.6+ MB


Here we will find the data dictionary that will help us understand the purpose each column has.  The details were created using information pulled from [Citi Bike's website](https://www.citibikenyc.com/system-data).

|Feature|Type|Description|
|---|---|---|
|tripduration|int|trip duration in seconds|
|starttime|object|start time of the trip, including the date|
|stoptime|object|stop time of the trip, including the date|
|start station id|int|the station id of where the trip started|
|start station name|object|the station name of where the trip started|
|start station latitude|float|the latitude coordinate of where the trip started|
|start station longitude|float|the longitude coordinate of where the trip started|
|end station id|int|the station id of where the trip ended|
|end station name|object|the station name of where the trip ended|
|end station latitude|float|the latitude coordinate of where the trip ended|
|end station longitude|float|the longitude coordinate of where the trip ended|
|bikeid|int|the id number of the bike used during the trip|
|usertype|object|Customer = 24-hour pass or 3-day pass user; Subscriber = Annual Member|
|birth year|float|birth year of user on the trip|
|gender|int| 0=unknown; 1=male; 2=female|

Now that we have formatted all of our data properly, the next section will be dedicated to building out a database in PostgreSQL.

---
### 03. Building a PostgreSQL Database

Here, we will establish a connection with PostgreSQL and create a new database.

In [14]:
# Connect to PostgreSQL
connection = psycopg2.connect("user=postgres password='password'");
connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT);

# Obtain a DB Cursor
cursor = connection.cursor();
db_name = "citibike_data";

# Create DB in PostgreSQL
create_database = f"CREATE DATABASE {db_name};"
cursor.execute(create_database);
print("Database created successfully in PostgreSQL ")

# Closing PostgreSQL connection
if(connection):
    cursor.close()
    connection.close()
    print("PostgreSQL connection is closed")

Database created successfully in PostgreSQL 
PostgreSQL connection is closed


Next, we'll create a table for the Jersey City data and then populate it with the data found in `jc_trip_data.csv`.

In [16]:
# Building a table for Jersey City data
try:
    connection = psycopg2.connect(user = 'postgres',
                                  password = 'password',
                                  host = '127.0.0.1',
                                  port = '5432',
                                  database = 'citibike_data')

    cursor = connection.cursor()
    
    # Some data types have been amended below to account for all data in the csv (i.e. blank cells)
    create_table_query = '''CREATE TABLE jersey_city(
                         trip_duration INT,
                         start_time TIMESTAMP,
                         stop_time TIMESTAMP,
                         start_station_id INT,
                         start_station_name TEXT,
                         start_station_latitude FLOAT,
                         start_station_longitude FLOAT,
                         end_station_id INT,
                         end_station_name TEXT,
                         end_station_latitude FLOAT,
                         end_station_longitude FLOAT,
                         bike_id INT,
                         user_type TEXT,
                         birth_year TEXT,
                         gender INT); '''
    
    cursor.execute(create_table_query)
    connection.commit()
    print("Table created successfully in PostgreSQL ")

except (Exception, psycopg2.DatabaseError) as error :
    print ("Error while creating PostgreSQL table:", error)
finally:
    #closing database connection.
        if(connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")

Table created successfully in PostgreSQL 
PostgreSQL connection is closed


Now that the table has been created, there are multiple ways to upload our csv data into the database.  One method is using `INSERT` from the `psycopg2` library.  Using this method is slightly inefficient, since we will ultimately have to loop through each row in the csv file.  A better way is to use `copy_from`.  With `copy_from`, we do not need to itereate through each row in the csv file - which is convenient, as the JC data is over 1.4 million rows - 1.4 million rows that we now don't have to loop through.

In [18]:
# Populating the Jersey City table
try:
    connection = psycopg2.connect(user = 'postgres',
                                  password = 'password',
                                  host = '127.0.0.1',
                                  port = '5432',
                                  database = 'citibike_data')

    cursor = connection.cursor()
    
    with open('../data/jc_trip_data.csv', 'r') as data:
        next(data) # Skip the header row
        cursor.copy_from(data, 'jersey_city', sep=',')
        
    connection.commit()
    
    print("Table updated successfully in PostgreSQL ")

except (Exception, psycopg2.DatabaseError) as error :
    print ("Error while updating PostgreSQL table:", error)
    
finally:
        if(connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")

Table updated successfully in PostgreSQL 
PostgreSQL connection is closed


So we have created, and populated, our PostgreSQL database.  The next notebook will be dedicated to creating queries to pull data that will help us generate useful insight into the world of Citi Bike in Jersey City.

---